In [1]:
from anthropic import Anthropic
from dotenv import load_dotenv
import os

# Load .env file
load_dotenv()

# Get key from environment
api_key = os.getenv("ANTHROPIC_API_KEY")

# Initialize client
client = Anthropic(api_key=api_key)

# Send a simple prompt
response = client.messages.create(
    model="claude-3-7-sonnet-20250219",
    max_tokens=200,
    messages=[
        {"role": "user", "content": "Say hello"}
    ]
)

print(response.content[0].text)


Hello! How can I assist you today?


In [2]:
from pathlib import Path
import pandas as pd, torch, os, gc
from interplm.sae.inference import load_sae_from_hf
import matplotlib.pyplot as plt
import numpy as np

DEVICE="cuda"
DTYPE=torch.float16

DATA_DIR = Path("esm_sae_results"); DATA_DIR.mkdir(exist_ok=True)
SEQUENCES_DIR = Path("/home/ec2-user/SageMaker/InterPLM/data/uniprot/subset_25k.csv")
# ANNOTATIONS_DIR = Path("uniprotkb_swissprot_annotations.tsv.gz")
ANNOTATIONS_DIR = Path("/home/ec2-user/SageMaker/InterPLM/uniprotkb_swissprot_annotations.tsv.gz")


/home/ec2-user/SageMaker/.cs/conda/envs/interplm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("facebook/esm2_t33_650M_UR50D", do_lower_case=False)
model     = AutoModel.from_pretrained("facebook/esm2_t33_650M_UR50D",
                                        output_hidden_states=True).to(DEVICE).eval()

# Make sure the SAE you load matches the *plm_model* and *plm_layer* you want to use
plm_model = "esm2-650m"   # matches your checkpoint naming
plm_layer = 24            # <= MUST match esm_layer_sel
sae = load_sae_from_hf(plm_model=plm_model, plm_layer=plm_layer).to(DEVICE).eval()


Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t33_650M_UR50D and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
import glob
features_all = pd.read_pickle("features_all.pkl")
features_all.shape


(40000, 6)

In [5]:
features_all.head()

,uniprot_id,length,features,max_activation,n_active_features,reconstruction_mse
0,Q9GL23,50,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.002...",1.265625,1876,45.198380
1,Q6GZU6,50,"[0.00023197175, 0.0, 0.0, 0.0, 0.0013056946, 0...",0.843262,2168,13.467114
2,P9WJG6,50,"[0.0, 0.00057144166, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.935059,1740,12.720748
3,P18924,51,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.000...",0.956543,1799,11.394856
4,Q08076,52,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.000...",1.139648,1772,24.694654


In [6]:
annotations_df = pd.read_csv(ANNOTATIONS_DIR, sep="\t", compression="gzip")

In [7]:
annotations_df.head()

,Entry,Reviewed,Protein names,Length,Sequence,EC number,Active site,Binding site,Cofactor,Disulfide bond,...,Helix,Turn,Beta strand,Coiled coil,Domain [CC],Compositional bias,Domain [FT],Motif,Region,Zinc finger
0,A0A009IHW8,reviewed,2' cyclic ADP-D-ribose synthase AbTIR (2'cADPR...,269,MSLEQKKGADIISKILQIQNSIGKTTSPSTLKTKLSEISRKEQENA...,3.2.2.-; 3.2.2.6,"ACT_SITE 208; /evidence=""ECO:0000255|PROSITE-P...","BINDING 143; /ligand=""NAD(+)""; /ligand_id=""ChE...",NaN,NaN,...,"HELIX 143..145; /evidence=""ECO:0007829|PDB:7UW...","TURN 146..149; /evidence=""ECO:0007829|PDB:7UWG...","STRAND 135..142; /evidence=""ECO:0007829|PDB:7U...","COILED 31..99; /evidence=""ECO:0000255""",DOMAIN: The TIR domain mediates NAD(+) hydrola...,NaN,"DOMAIN 133..266; /note=""TIR""; /evidence=""ECO:0...",NaN,NaN,NaN
1,A0A023I7E1,reviewed,"Glucan endo-1,3-beta-D-glucosidase 1 (Endo-1,3...",796,MRFQVIVAAATITMITSYIPGVASQSTSDGDDLFVPVSNFDPKSIF...,3.2.1.39,"ACT_SITE 500; /evidence=""ECO:0000255|PROSITE-P...","BINDING 504; /ligand=""(1,3-beta-D-glucosyl)n"";...",NaN,NaN,...,"HELIX 42..44; /evidence=""ECO:0007829|PDB:4K35""...","TURN 287..289; /evidence=""ECO:0007829|PDB:4K35...","STRAND 56..58; /evidence=""ECO:0007829|PDB:4K35...",NaN,NaN,NaN,"DOMAIN 31..759; /note=""GH81""; /evidence=""ECO:0...",NaN,"REGION 31..276; /note=""beta-sandwich subdomain...",NaN
2,A0A024B7W1,reviewed,Genome polyprotein [Cleaved into: Capsid prote...,3423,MKNPKKKSGGFRIVNMLKRGVARVSPFGGLKRLPAGLLLGHGPIRM...,2.1.1.56; 2.1.1.57; 2.7.7.48; 3.4.21.91; 3.6.1...,"ACT_SITE 1553; /note=""Charge relay system; for...","BINDING 1696..1703; /ligand=""ATP""; /ligand_id=...",NaN,"DISULFID 350..406; /evidence=""ECO:0000250|UniP...",...,"HELIX 222..225; /evidence=""ECO:0007829|PDB:6CO...","TURN 237..241; /evidence=""ECO:0007829|PDB:6CO8...","STRAND 234..236; /evidence=""ECO:0007829|PDB:6C...",NaN,DOMAIN: [Small envelope protein M]: The transm...,NaN,"DOMAIN 1503..1680; /note=""Peptidase S7""; /evid...","MOTIF 1787..1790; /note=""DEAH box""; /evidence=...","REGION 1..25; /note=""Disordered""; /evidence=""E...",NaN
3,A0A024RXP8,reviewed,"Exoglucanase 1 (EC 3.2.1.91) (1,4-beta-cellobi...",514,MYRKLAVISAFLATARAQSACTLQSETHPPLTWQKCSSGGTCTQQT...,3.2.1.91,"ACT_SITE 229; /note=""Nucleophile""; /evidence=""...",NaN,NaN,"DISULFID 21..89; /evidence=""ECO:0000250|UniPro...",...,NaN,NaN,NaN,NaN,DOMAIN: The enzyme consists of two functional ...,"COMPBIAS 401..437; /note=""Polar residues""; /ev...","DOMAIN 478..514; /note=""CBM1""; /evidence=""ECO:...",NaN,"REGION 18..453; /note=""Catalytic""; /evidence=""...",NaN
4,A0A024SC78,reviewed,Cutinase (EC 3.1.1.74),248,MRSLAILTTLLAGHAFAYPKPAPQSVNRRDWPSINEFLSELAKVMP...,3.1.1.74,"ACT_SITE 164; /note=""Nucleophile""; /evidence=""...",NaN,NaN,"DISULFID 55..91; /evidence=""ECO:0000269|PubMed...",...,"HELIX 51..69; /evidence=""ECO:0007829|PDB:4PSC""...","TURN 94..100; /evidence=""ECO:0007829|PDB:4PSC""...","STRAND 48..50; /evidence=""ECO:0007829|PDB:4PSC...",NaN,"DOMAIN: In contract to classical cutinases, po...",NaN,NaN,NaN,"REGION 31..70; /note=""Lid covering the active ...",NaN


In [8]:
# import numpy as np
# import pandas as pd
# import random

# # Parameters
# N_FEATURES = 1200
# BINS = np.arange(0, 1.1, 0.1)

# # Randomly select feature ids
# all_feature_ids = list(range(len(features_all.iloc[0].features)))
# print("num features", len(all_feature_ids))
# selected_features = random.sample(all_feature_ids, N_FEATURES)

# print(f"Selected {len(selected_features)} features out of {len(all_feature_ids)}")

# # Build dataset for each feature
# feature_datasets = {}

# # Predefine bin labels
# bin_labels = [f"{BINS[i]:.1f}-{BINS[i+1]:.1f}" for i in range(len(BINS)-1)]

# for fid in selected_features:
#     # Extract activations for this feature
#     activations = [f[fid] for f in features_all["features"]]
#     df = pd.DataFrame({
#         "uniprot_id": features_all["uniprot_id"],
#         "activation": activations
#     })

#     # Assign bins
#     df["bin"] = pd.cut(df["activation"], bins=BINS, labels=bin_labels, include_lowest=True)

#     sampled = []

#     # Sample proteins per bin
#     for b in df["bin"].dropna().unique():
#         bin_df = df[df["bin"] == b]
#         n = 10 if b == "0.9-1.0" else 2
#         sampled.extend(bin_df.sample(min(len(bin_df), n), random_state=42).to_dict(orient="records"))

#     # Add 10 random zero-activation proteins 
#     zero_df = df[df["activation"] == 0.0]
#     if len(zero_df) > 0:
#         sampled.extend(zero_df.sample(min(len(zero_df), 10), random_state=42).to_dict(orient="records"))

#     # Merge with metadata from annotations_df
#     sampled_df = pd.DataFrame(sampled)
#     merged = sampled_df.merge(annotations_df, left_on="uniprot_id", right_on="Entry", how="left")

#     feature_datasets[fid] = merged

# # Example feature dataset
# example_fid = selected_features[0]
# feature_datasets[example_fid].head()


## Normalize features

In [9]:
import numpy as np

#Stakc into [num_proteins, num_features]
X = np.vstack(features_all['features'].values) #Shape (N, F)

#Max activation per feature across all proteins
max_per_feature = X.max(axis=0) # shape: (F,)
eps = 1e-12
max_safe = np.where(max_per_feature > 0, max_per_feature, eps)
#Normalize
X_norm = X / max_safe

#Save back
features_all = features_all.copy()
features_all["features_norm"] = [row for row in X_norm]
print("Original max activation (feature 0):", X[:,0].max())
print("Normalized max activation (feature 0):", X_norm[:,0].max())

Original max activation (feature 0): 0.02611415
Normalized max activation (feature 0): 1.0


In [10]:
import numpy as np
import pandas as pd
import random

# Parameters
N_FEATURES = 10240
BINS = np.arange(0, 1.1, 0.1)

# Randomly select feature ids
# all_feature_ids = list(range(len(features_all.iloc[0].features)))
# print("num features", len(all_feature_ids))
# selected_features = random.sample(all_feature_ids, N_FEATURES)

# print(f"Selected {len(selected_features)} features out of {len(all_feature_ids)}")

# # Build dataset for each feature
# feature_datasets = {}

# # Predefine bin labels
# bin_labels = [f"{BINS[i]:.1f}-{BINS[i+1]:.1f}" for i in range(len(BINS)-1)]

# for fid in selected_features:
#     # Extract activations for this feature
#     activations = [f[fid] for f in features_all["features_norm"]]
#     df = pd.DataFrame({
#         "uniprot_id": features_all["uniprot_id"],
#         "activation": activations
#     })

#     # Assign bins
#     df["bin"] = pd.cut(df["activation"], bins=BINS, labels=bin_labels, include_lowest=True)

#     sampled = []

#     # Sample proteins per bin
#     for b in df["bin"].dropna().unique():
#         bin_df = df[df["bin"] == b]
#         n = 10 if b == "0.9-1.0" else 2
#         sampled.extend(bin_df.sample(min(len(bin_df), n), random_state=42).to_dict(orient="records"))

#     # Add 10 random zero-activation proteins 
#     zero_df = df[df["activation"] == 0.0]
#     if len(zero_df) > 0:
#         sampled.extend(zero_df.sample(min(len(zero_df), 10), random_state=42).to_dict(orient="records"))

#     # Merge with metadata from annotations_df
#     sampled_df = pd.DataFrame(sampled)
#     merged = sampled_df.merge(annotations_df, left_on="uniprot_id", right_on="Entry", how="left")

#     feature_datasets[fid] = merged

# # Example feature dataset
# example_fid = selected_features[0]
# feature_datasets[example_fid].head()


In [11]:
# import pickle

# # Suppose feature_datasets is your dict of DataFrames
# with open("feature_datasets.pkl", "wb") as f:
#     pickle.dump(feature_datasets, f)

In [12]:
import pickle
with open("feature_datasets.pkl", "rb") as f:
    feature_datasets = pickle.load(f)


## Add Amino Acid Indices and activations for better LLM annotations

In [13]:
from utils import extract_esm_features_batch

@torch.no_grad()
def extract_sae_features(hidden_states: torch.Tensor, sae):
    """
    Pass ESM hidden states through the Sparse Autoencoder (SAE).

    Args
    ----
    hidden_states : torch.Tensor
        Shape [B, L, d] or [L, d].
        - B = batch size (optional if unsqueezed)
        - L = sequence length
        - d = ESM embedding dimension (e.g., 1280 for esm2_t33_650M)

    Returns
    -------
    sae_features : torch.Tensor
        Shape [B, L, F]
        Sparse latent features per residue.
        F = number of SAE dictionary atoms / features.

    recon : torch.Tensor
        Shape [B, L, d]
        Reconstructed embeddings in token space.

    error : torch.Tensor
        Shape [B, L, d]
        Residual = hidden_states - recon
    """
    if hidden_states.dim() == 2:          # [L, d]
        hidden_states = hidden_states.unsqueeze(0)  # → [1, L, d]
    x = hidden_states.to(torch.float32)      # <- ensure fp32 for SAE

    # SAE should have encode() and decode() that operate on last dimension
    sae_features = sae.encode(x)     # [B, L, F]
    recon        = sae.decode(sae_features)      # [B, L, d]
    error        = hidden_states - recon         # [B, L, d]

    return sae_features, recon, error

#Config for extracting activated positions

TOP_K = 8 # How many positions to record per protein
MIN_ACT = 0.0 #Only consider positions with activation > MIN_ACT
BATCH_SIZE = 16 #For ESM/SAE Inference


def _batched(iterable, n):
    """Yield Successive n-sized chunks from iterable
    """
    it = list(iterable)
    for i in range(0, len(it), n):
        yield it[i:i+n]

import numpy as np
import torch
from typing import List, Tuple, Optional, Literal

TOP_K = 8
MIN_ACT = 0.0
BATCH_SIZE = 16

def _batched(iterable, n):
    it = list(iterable)
    for i in range(0, len(it), n):
        yield it[i:i+n]

def _normalize_1d(
    x: np.ndarray,
    mode: Literal["seq_max","feature_global_max","zscore","none"] = "seq_max",
    global_max: Optional[float] = None,
    eps: float = 1e-8
) -> np.ndarray:
    """
    Normalize a 1D activation vector x (valid positions only).
    """
    if mode == "none":
        return x.copy()

    if mode == "feature_global_max":
        if global_max is None or global_max <= eps:
            # fallback to seq_max if global not available/safe
            mode = "seq_max"
        else:
            return x / (global_max + eps)

    if mode == "seq_max":
        m = np.max(x) if x.size else 0.0
        return x / (m + eps)

    if mode == "zscore":
        mu = float(np.mean(x)) if x.size else 0.0
        sd = float(np.std(x)) if x.size else 0.0
        return (x - mu) / (sd + eps)

    # Fallback (shouldn't hit)
    return x.copy()

@torch.no_grad()
def compute_activated_positions_for_feature(
    fid: int,
    seqs: List[str],
    batch_size: int = BATCH_SIZE,
    max_per_feature: Optional[np.ndarray] = None,
    norm_mode: Literal["seq_max","feature_global_max","zscore","none"] = "seq_max",
    top_k: int = TOP_K,
    min_act: float = MIN_ACT,
    device: str = "cuda"
) -> Tuple[List[List[int]], List[List[str]], List[List[float]], List[List[float]]]:
    """
    For a list of sequences and a single SAE feature id (fid),
    return per-sequence top-K activated residue indices, AA identities,
    normalized scores (per selected position), and raw scores.

    Returns
    -------
    all_indices : List[List[int]]
        Per-sequence Top-K indices (0-based).
    all_aas : List[List[str]]
        Per-sequence amino acids at those indices.
    norm_vals : List[List[float]]
        Per-sequence normalized activations aligned with Top-K order.
    raw_vals : List[List[float]]
        Per-sequence raw activations aligned with Top-K order.
    """
    all_indices: List[List[int]] = []
    all_aas: List[List[str]] = []
    norm_vals: List[List[float]] = []
    raw_vals: List[List[float]] = []

    # Optional global max for this feature (for 'feature_global_max' mode)
    global_max = None
    if norm_mode == "feature_global_max" and max_per_feature is not None:
        if 0 <= fid < len(max_per_feature):
            gm = float(max_per_feature[fid])
            global_max = gm if np.isfinite(gm) else None

    for chunk in _batched(seqs, batch_size):
        # ESM -> SAE: token representations and mask
        token_reps, attn_mask = extract_esm_features_batch(
            chunk, layer_sel=plm_layer, device=device, model=model, tokenizer=tokenizer
        )  # token_reps: [B, L, H], attn_mask: [B, L] bool
        sae_feats, _, _ = extract_sae_features(token_reps, sae)  # [B, L, F]

        # Select feature channel -> [B, L] on CPU
        feat_act = sae_feats[..., fid].float().cpu()
        mask = attn_mask.cpu()  # [B, L] bool

        for seq, act_row, m in zip(chunk, feat_act, mask):
            L = int(m.sum().item())  # valid residues
            act_valid = act_row[:L].numpy() if L > 0 else np.array([], dtype=np.float32)

            # positions above threshold
            valid_idx = np.where(act_valid > min_act)[0]
            if valid_idx.size == 0:
                all_indices.append([])
                all_aas.append([])
                norm_vals.append([])
                raw_vals.append([])
                continue

            # sort by activation desc within valid subset and take top_k
            order = np.argsort(-act_valid[valid_idx])
            chosen_local = valid_idx[order[:top_k]].tolist()

            # raw values for the chosen positions
            chosen_raw = act_valid[chosen_local]

            # normalization (done on the full valid slice, then gather chosen)
            norm_full = _normalize_1d(
                act_valid, mode=norm_mode, global_max=global_max
            )
            chosen_norm = norm_full[chosen_local]

            # map to AAs (defensive indexing)
            aas = [seq[i] if i < len(seq) else "X" for i in chosen_local]

            all_indices.append(chosen_local)
            all_aas.append(aas)
            norm_vals.append([float(v) for v in chosen_norm])
            raw_vals.append([float(v) for v in chosen_raw])

        # free tensors early
        del token_reps, sae_feats, feat_act, attn_mask
        torch.cuda.empty_cache()

    return all_indices, all_aas, norm_vals, raw_vals


## Using ray so this doesn't take an hour and a half

In [21]:
import os
import importlib, utils
import ray_based_processing
importlib.reload(utils)
importlib.reload(ray_based_processing)

from utils import _batched

## Integrate with feature_datasets dict

In [31]:
# ray_feature_activation.py
import os
import ray
import torch
import pandas as pd
import numpy as np
from pathlib import Path
from typing import Dict, List, Optional, Literal, Tuple
from transformers import AutoTokenizer, AutoModel
from interplm.sae.inference import load_sae_from_hf

# === Your existing helpers (import these from your utils if already defined) ===
from utils import extract_esm_features_batch, _batched, _normalize_1d

# ----------------------------
# Config
# ----------------------------
DATA_DIR = Path("esm_sae_results"); DATA_DIR.mkdir(exist_ok=True, parents=True)
os.environ.setdefault("HF_HOME", str(DATA_DIR / "hf_cache"))
os.environ.setdefault("TRANSFORMERS_CACHE", str(DATA_DIR / "hf_cache"))
os.environ.setdefault("PYTORCH_CUDA_ALLOC_CONF", "expandable_segments:True,max_split_size_mb:128")

# Ray init
ray.shutdown()
ray.init(ignore_reinit_error=True)

# GPU world size
N_GPUS = int(ray.available_resources().get("GPU", 0))
WORLD_SIZE = max(1, min(8, N_GPUS))

# ----------------------------
# Save your feature_datasets to disk to avoid shipping a giant object
# feature_datasets: Dict[int, pd.DataFrame] must already exist in memory
# ----------------------------
# Example:
#   feature_datasets: Dict[int, pd.DataFrame]  # keys=fids; each df has at least a 'Sequence' column
#   max_safe: Optional[np.ndarray]             # global max per feature if using "feature_global_max"
#   BATCH_SIZE, TOP_K, MIN_ACT, NORM_MODE, ESM_LAYER_SEL defined

FEATURE_PICKLE = DATA_DIR / "feature_datasets.pkl"
META_PICKLE    = DATA_DIR / "feature_meta.pkl"
ESM_LAYER_SEL=24
# Write once here in the driver
# (We store (fids list, shapes) separately if you’d like; for now we just persist the dict)
pd.to_pickle(feature_datasets, FEATURE_PICKLE)
pd.to_pickle(
    dict(
        all_fids=list(feature_datasets.keys()),
        norm_mode=NORM_MODE,
        top_k=TOP_K,
        min_act=MIN_ACT,
        batch_size=BATCH_SIZE,
        plm_layer=(ESM_LAYER_SEL if isinstance(ESM_LAYER_SEL, int) else 24),
        max_per_feature=np.asarray(max_safe) if max_safe is not None else None,
    ),
    META_PICKLE,
)

# ----------------------------
# Core compute function (uses passed-in model/tokenizer/sae) — same logic you posted
# ----------------------------
@torch.no_grad()
def _compute_activated_positions_for_feature_wrapped(
    fid: int,
    seqs: List[str],
    *,
    batch_size: int,
    max_per_feature: Optional[np.ndarray] = None,
    norm_mode: Literal["seq_max", "feature_global_max", "zscore", "none"] = "seq_max",
    top_k: int = 5,
    min_act: float = 0.0,
    device: str = "cuda:0",
    plm_layer: int = 24,
    model=None,
    tokenizer=None,
    sae=None,
) -> Tuple[List[List[int]], List[List[str]], List[List[float]], List[List[float]]]:
    all_indices: List[List[int]] = []
    all_aas: List[List[str]] = []
    norm_vals: List[List[float]] = []
    raw_vals: List[List[float]] = []

    global_max = None
    if norm_mode == "feature_global_max" and max_per_feature is not None:
        if 0 <= fid < len(max_per_feature):
            gm = float(max_per_feature[fid])
            global_max = gm if np.isfinite(gm) else None

    for chunk in _batched(seqs, batch_size):
        token_reps, attn_mask = extract_esm_features_batch(
            chunk,
            layer_sel=plm_layer,
            device=device,
            model=model,
            tokenizer=tokenizer,
        )

        sae_feats, _, _ = extract_sae_features(token_reps, sae)  # [B, L, F]
        feat_act = sae_feats[..., fid].float().cpu()
        mask = attn_mask.cpu()

        for seq, act_row, m in zip(chunk, feat_act, mask):
            L = int(m.sum().item())
            act_valid = act_row[:L].numpy() if L > 0 else np.array([], dtype=np.float32)

            valid_idx = np.where(act_valid > min_act)[0]
            if valid_idx.size == 0:
                all_indices.append([])
                all_aas.append([])
                norm_vals.append([])
                raw_vals.append([])
                continue

            order = np.argsort(-act_valid[valid_idx])
            chosen_local = valid_idx[order[:top_k]].tolist()

            chosen_raw = act_valid[chosen_local]
            norm_full = _normalize_1d(act_valid, mode=norm_mode, global_max=global_max)
            chosen_norm = norm_full[chosen_local]

            aas = [seq[i] if i < len(seq) else "X" for i in chosen_local]

            all_indices.append(chosen_local)
            all_aas.append(aas)
            norm_vals.append([float(v) for v in chosen_norm])
            raw_vals.append([float(v) for v in chosen_raw])

        # clean up
        del token_reps, sae_feats, feat_act, attn_mask
        torch.cuda.empty_cache()

    return all_indices, all_aas, norm_vals, raw_vals

# ----------------------------
# Ray worker (Actor) to reuse model/tokenizer/sae across many features
# ----------------------------
@ray.remote(num_gpus=1, num_cpus=0, max_retries=1)
class FeatureWorker:
    def __init__(self, amp_dtype: torch.dtype = torch.float16, plm_layer: int = 24):
        from tqdm.auto import tqdm  # local import OK
        self.tqdm = tqdm

        self.device = torch.device("cuda:0")
        self.dtype  = amp_dtype
        self.plm_layer = plm_layer

        # Load ESM
        self.tokenizer = AutoTokenizer.from_pretrained("facebook/esm2_t33_650M_UR50D", do_lower_case=False)
        self.model = AutoModel.from_pretrained(
            "facebook/esm2_t33_650M_UR50D",
            output_hidden_states=True,
            torch_dtype=self.dtype,
            low_cpu_mem_usage=True,
        ).to(self.device).eval()

        # Load SAE
        self.sae = load_sae_from_hf(plm_model="esm2-650m", plm_layer=self.plm_layer).to(self.device).eval()

    def process_shard(
        self,
        rank: int,
        world_size: int,
        feature_pickle_path: str,
        meta_pickle_path: str,
        out_prefix: str = "feature_acts",
    ) -> List[str]:
        # Load inputs
        fd: Dict[int, pd.DataFrame] = pd.read_pickle(feature_pickle_path)
        meta = pd.read_pickle(meta_pickle_path)

        all_fids: List[int] = meta["all_fids"]
        norm_mode: str      = meta["norm_mode"]
        top_k: int          = meta["top_k"]
        min_act: float      = meta["min_act"]
        batch_size: int     = meta["batch_size"]
        plm_layer: int      = meta["plm_layer"]
        max_per_feature     = meta["max_per_feature"]

        # This worker’s slice of features
        shard_fids = all_fids[rank::world_size]

        paths = []
        pbar = self.tqdm(total=len(shard_fids), desc=f"GPU{rank}", position=rank, leave=True)

        for fid in shard_fids:
            df = fd[fid].copy()
            if "Sequence" not in df.columns:
                raise KeyError("Expected a 'Sequence' column in feature_datasets[fid].")

            seqs = df["Sequence"].astype(str).fillna("").tolist()

            idx_lists, aa_lists, norm_vals, raw_vals = _compute_activated_positions_for_feature_wrapped(
                fid=fid,
                seqs=seqs,
                batch_size=batch_size,
                max_per_feature=max_per_feature,
                norm_mode=norm_mode,
                top_k=top_k,
                min_act=min_act,
                device=str(self.device),
                plm_layer=plm_layer,
                model=self.model,
                tokenizer=self.tokenizer,
                sae=self.sae,
            )

            # Attach outputs
            df["activated_indices"] = idx_lists
            df["activated_aas"] = aa_lists
            df["seq_max_activation_norm"] = norm_vals
            df["activated_indices_str"] = df["activated_indices"].apply(lambda xs: ",".join(map(str, xs)) if xs else "")
            df["activated_aas_str"] = df["activated_aas"].apply(lambda xs: ",".join(xs) if xs else "")
            df["seq_max_activation_norm_str"] = df["seq_max_activation_norm"].apply(
                lambda xs: ",".join(map(str, xs)) if xs else ""
            )

            # Persist per-fid (nice for resuming/merging)
            out_path = DATA_DIR / f"{out_prefix}_fid{fid}_rank{rank}.pkl"
            df.to_pickle(out_path)
            paths.append(str(out_path))

            pbar.update(1)

        # finish this worker shard bundle as well (optional consolidated file)
        shard_bundle = DATA_DIR / f"{out_prefix}_rank{rank}.final.pkl"
        pd.to_pickle({"rank": rank, "paths": paths}, shard_bundle)

        pbar.close()
        return [str(p) for p in paths] + [str(shard_bundle)]

# ----------------------------
# Kick off workers
# ----------------------------


futs = []
for r in range(WORLD_SIZE):
    actor = FeatureWorker.remote(amp_dtype=torch.float16, plm_layer=(ESM_LAYER_SEL if isinstance(ESM_LAYER_SEL, int) else 24))
    futs.append(actor.process_shard.remote(
        rank=r,
        world_size=WORLD_SIZE,
        feature_pickle_path=str(FEATURE_PICKLE),
        meta_pickle_path=str(META_PICKLE),
        out_prefix="feature_acts",
    ))

out_paths_per_rank = ray.get(futs)
flat_paths = [p for sub in out_paths_per_rank for p in sub]
print(f"[done] wrote {len(flat_paths)} files")


2025-09-13 19:55:43,593	INFO worker.py:1951 -- Started a local Ray instance.


ValueError: Invalid option keyword max_retries for actors. Valid ones are ['label_selector', 'accelerator_type', 'memory', 'name', 'num_cpus', 'num_gpus', 'object_store_memory', 'placement_group', 'placement_group_bundle_index', 'placement_group_capture_child_tasks', 'resources', 'runtime_env', 'scheduling_strategy', '_metadata', 'enable_task_events', '_labels', 'concurrency_groups', 'enable_tensor_transport', 'lifetime', 'max_concurrency', 'max_restarts', 'max_task_retries', 'max_pending_calls', 'namespace', 'get_if_exists', 'allow_out_of_order_execution'].

In [ ]:
# Suppose feature_datasets is your dict of DataFrames
with open("feature_datasets_with_amino_acids.pkl", "wb") as f:
    pickle.dump(feature_datasets, f)

In [33]:
import boto3
# --- Setup (once per notebook) ---
# pip install anthropic python-dotenv
from anthropic import Anthropic
from dotenv import load_dotenv
import os, time, json, math, textwrap
import numpy as np
import pandas as pd
from typing import Dict

# Load .env (expects ANTHROPIC_API_KEY=...)
load_dotenv()
client = Anthropic(api_key=os.getenv("ANTHROPIC_API_KEY"))

# --- Config ---
MODEL_NAME = "claude-3-5-sonnet-20240620"
MAX_TOKENS = 800  # enough for description + summary
TEMPERATURE = 0.0 # deterministic
CHECKPOINT_EVERY = 50
OUTPUT_PATH = "claude_feature_annotations.csv"

# Columns to show Claude (customize as you like)
# We'll include what exists; missing columns are auto-dropped
PREFERRED_COLS = [
    # keys/ids
    "uniprot_id", "Entry", "Protein names",
    # size/sequence shape
    "Length",
    # functional annotations
    "EC number", "Active site", "Binding site", "Cofactor", "Disulfide bond",
    "Helix", "Turn", "Beta strand", "Coiled coil",
    "Domain [CC]", "Compositional bias", "Domain [FT]", "Motif", "Region", "Zinc finger",
    # your per-feature fields
    "activation", "bin",
    # optional (only used if present)
    "activated_indices", "activated_aas"
]

# Limit rows/cols so the table fits comfortably in context
MAX_ROWS = 80   # you can raise/lower if you hit token limits
TRUNCATE_STR_LEN = 120  # truncate long text fields so tables tay compact


def _coerce_and_trim_cols(df: pd.DataFrame, cols: list[str]) -> pd.DataFrame:
    """Select existing columns, stringify, and truncate long strings so the table is compact."""
    use_cols = [c for c in cols if c in df.columns]
    if not use_cols:
        # Fallback: show whatever is available
        use_cols = list(df.columns)

    out = df[use_cols].copy()

    # Coerce to string and truncate long values
    for c in use_cols:
        out[c] = out[c].astype(str).str.replace(r"\s+", " ", regex=True)
        out[c] = out[c].apply(lambda s: s[:TRUNCATE_STR_LEN] + "…" if len(s) > TRUNCATE_STR_LEN else s)

    # Keep only first MAX_ROWS to control token usage
    return out.head(MAX_ROWS)


PROMPT_TEMPLATE = """Generate description and summary
Analyze this protein dataset to determine what predicts the ’Maximum activation value’ and ‘Amino acids of
highest activated indices in protein’ columns. This description should be as concise as possible but sufficient to
predict these two columns on held-out data given only the description and the rest of the protein metadata
provided. The feature could be specific to a protein family, a structural motif, a sequence motif, a functional
role, etc. These WILL be used to predict how much unseen proteins are activated by the feature so only
highlight relevant factors for this.

Focus on:
• Properties of proteins from the metadata that are associated with high vs medium vs low activation.
• Where in the protein sequence activation occurs (in relation to the protein sequence, length, structure,
  or other properties)
• What functional annotations (binding sites, domains, etc.) and amino acids are present at or near the
  activated positions
• This description that will be used to help predict missing activation values should start with:
  “The activation patterns are characterized by:”

Then, in 1 sentence, summarize what biological feature or pattern this neural network activation is detecting.
This concise summary should start with “The feature activates on”.

Protein record:
{TABLE}
"""

def build_prompt(table_df: pd.DataFrame) -> str:
    table_md = table_df.to_markdown(index=False)
    return PROMPT_TEMPLATE.replace("{TABLE}", table_md)

# Configure once
BEDROCK_REGION = "us-east-1"
MODEL_ID = "us.anthropic.claude-3-opus-20240229-v1:0"  # change if you prefer another Claude on Bedrock

_bedrock = boto3.client("bedrock-runtime", region_name=BEDROCK_REGION)


def call_claude(prompt: str) -> str:
    """Call Claude, return raw text."""

    #Build Bedrock/Anthropic messages pyalod
    messages = [
    {
        "role": "user",
        "content": [{"type":"text", "text": prompt}]
    }
    ]

    body = {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": MAX_TOKENS,
        "temperature" : TEMPERATURE,
        "messages": messages
    }

    resp = _bedrock.invoke_model(
        modelId=MODEL_ID,
        body=json.dumps(body),
        contentType = "application/json",
        accept="application/json",
    )
    payload = json.loads(resp["body"].read())

    #Concatenate all text content blocks(CLaude may return multiple)
    text_parts = []
    for part in payload.get("content", []):
        if part.get("type") == "text":
            text-parts.append(part.get("text", ""))
    return "".join(text_parts)

def parse_description_and_summary(text: str) -> dict:
    """
    Best-effort parse: extract the long description (must start with the required phrase)
    and the one-sentence summary (starts with 'The feature activates on').
    Falls back to raw if patterns aren’t found.
    """
    desc = ""
    summ = ""
    lines = [l.strip() for l in text.splitlines() if l.strip()]

    # Find the description block
    start_idx = None
    for i, l in enumerate(lines):
        if l.lower().startswith("the activation patterns are characterized by:"):
            start_idx = i
            break
    if start_idx is not None:
        # collect until we hit the summary or end
        buff = []
        for j in range(start_idx, len(lines)):
            if lines[j].lower().startswith("the feature activates on"):
                break
            buff.append(lines[j])
        desc = "\n".join(buff).strip()

    # Find the one-sentence summary
    for l in lines:
        if l.lower().startswith("the feature activates on"):
            # keep first sentence
            summ = l.split("\n")[0].strip()
            break

    return {
        "description": desc or "",
        "summary": summ or "",
        "raw": text.strip()
    }

# --- Main loop over feature datasets ---
# Expects: feature_datasets: Dict[int, pd.DataFrame]
results_rows = []

processed = 0
for fid, df in list(feature_datasets.items())[:3]:
    # Build a compact table for the model
    view = _coerce_and_trim_cols(df, PREFERRED_COLS)
    prompt = build_prompt(view)

    try:
        text = call_claude(prompt)
        parsed = parse_description_and_summary(text)
    except Exception as e:
        parsed = {"description": "", "summary": "", "raw": f"[ERROR] {e}"}

    results_rows.append({
        "feature_id": fid,
        "n_rows_shown": len(view),
        "description": parsed["description"],
        "summary": parsed["summary"],
        "raw_response": parsed["raw"],
    })

    processed += 1
    if processed % CHECKPOINT_EVERY == 0:
        pd.DataFrame(results_rows).to_parquet(OUTPUT_PATH, index=False)
        print(f"[checkpoint] saved {processed} → {OUTPUT_PATH}")

# Final save
df_results = pd.DataFrame(results_rows)
df_results.to_csv(OUTPUT_PATH, index=False)
print(f"[done] {len(df_results)} features → {OUTPUT_PATH}")
df_results.head()


In [23]:
results_rows = []
processed = 0

fid = 2167
df  = feature_datasets[fid]          # <- this is a DataFrame

view   = _coerce_and_trim_cols(df, PREFERRED_COLS)
prompt = build_prompt(view)

try:
    text   = call_claude(prompt)
    parsed = parse_description_and_summary(text)
except Exception as e:
    parsed = {"description": "", "summary": "", "raw": f"[ERROR] {e}"}

results_rows.append({
    "feature_id": fid,
    "n_rows_shown": len(view),
    "description": parsed["description"],
    "summary": parsed["summary"],
    "raw_response": parsed["raw"],
})

# Save
df_results = pd.DataFrame(results_rows)
df_results.to_csv(OUTPUT_PATH, index=False)
print(f"[done] {len(df_results)} features → {OUTPUT_PATH}")
display(df_results.head())


NameError: name '_coerce_and_trim_cols' is not defined

In [ ]:
#feature_datasets: Dict[int, pd.DataFrame] where each df includes 'Sequence'
from tqdm import tqdm
for fid, df in tqdm(list(feature_datasets.items()), total=len(feature_datasets), desc="Features"):
    #make a copy to avoid mutating original reference
    work = df.copy()

    # Ensure sequences are present and aligned
    if "Sequence" not in work.columns:
        # If your UniProt column is named differently, adjust here
        raise KeyError("Expected a 'Sequence' column in merged annotations.")
    seqs = work["Sequence"].astype(str).fillna("").tolist()

    idx_lists, aa_lists, norm_vals, raw_vals = compute_activated_positions_for_feature(fid, seqs, batch_size=BATCH_SIZE, max_per_feature = max_safe)

    #compute per-row activated positions for this feature
    work['activated_indices'] = idx_lists
    work['activated_aas'] = aa_lists
    work['seq_max_activation_norm'] = norm_vals

    # # (Optional) also add a compact string column for the prompt table
    work["activated_indices_str"] = work["activated_indices"].apply(lambda xs: ",".join(map(str, xs)) if xs else "")
    work["activated_aas_str"]     = work["activated_aas"].apply(lambda xs: ",".join(xs) if xs else "")
    work["seq_max_activation_norm_str"] = work["seq_max_activation_norm"].apply(lambda xs: ",".join(map(str, xs)) if xs else "")

    feature_datasets[fid] = work

first_fid = list(feature_datasets.keys())[0]
feature_datasets[first_fid][["uniprot_id","activation","seq_max_activation_norm", "activated_indices_str","activated_aas_str", "seq_max_activation_norm_str"]].head()

    

In [32]:
import boto3
import json

# Initialize the Bedrock client
bedrock = boto3.client(
    service_name="bedrock-runtime",
    region_name="us-east-1"  # Or your preferred region
)

# Prepare the request body
body = json.dumps({
    "anthropic_version": "bedrock-2023-05-31",
    "max_tokens": 256,
    "messages": [
        {
            "role": "user", 
            "content": "Hello, world"
        }
    ]
})

# Use the US cross-region inference profile
response = bedrock.invoke_model(
    body=body,
    modelId="us.anthropic.claude-3-opus-20240229-v1:0",  # Note the "us." prefix
    contentType="application/json",
    accept="application/json"
)

# Parse the response
response_body = json.loads(response.get("body").read())

# Extract the text from the response
if "content" in response_body:
    for content in response_body["content"]:
        if content["type"] == "text":
            print(content["text"])

Hello! How can I assist you today?
